In [1]:
import torch
import torch.nn as nn
import math
import numpy as np

np.random.seed(42)

In [2]:
class RequestType:
    def __init__(self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None):
        # distribution is 1x2 if elastic and 1x1 if static
        
        self.type = request_type
        self.bw = bandwidth
        self.service_rate = service_rate
        self.arrival_rate = arrival_rate
        self.source = source
        self.sink = sink
        self.distribution = distribution
        self.switch_rate = switch_rate

class Request:
    def __init__(self, request_type, service_time, arrival_time, source, sink, transfer_rate, distribution=None, parent_elastic=None, bw_dist=None):
        self.type = request_type
        self.service_time = service_time
        self.arrival_time = arrival_time
        self.source = source
        self.sink = sink
        self.bw = transfer_rate
        self.request_type = request_type
        self.parent_elastic = parent_elastic
        self.accepted = None
        self.path = None
        self.bw_dist = bw_dist
        
        if request_type == "elastic":
            self.distribution = distribution
            self.scale_requests = []
            
    def add_scale_request(self, req): 
        # we store related scale requests for elastic requests
        # not used if static request
        self.scale_requests.append(req)
            
    def get_encoding(self, nodes_in_environment):
        # as per our notes, this SHOULD return 1x5 tensor,
        # but we have one hot encodings INSIDE this tensor,
        # so we will flatten this and return, so the size will be
        # larger than 1x5
        
        # nodes_in_environment is a list of all the nodes in our graph
        # eg ["a", "b", "c"]
        
        # request is [one hot source, one hot destination, bw, service time, one hot type]
                
        one_hot_source = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.source)]), num_classes=len(nodes_in_environment)).flatten()
        one_hot_dest   = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.sink)]), num_classes=len(nodes_in_environment)).flatten()
    
        if self.request_type == "static" or self.request_type == "scale":
            one_hot_type = torch.tensor([1, 0])
        elif self.request_type == "elastic":
            one_hot_type = torch.tensor([0, 1])
            
        encoding = torch.cat([one_hot_source, 
                             one_hot_dest,
                             torch.tensor([self.bw]), 
                             torch.tensor([self.service_time]),
                             one_hot_type])
        
        return encoding

In [3]:
class Link:
    def __init__(self, node_1, node_2, bw_capacity):
        self.serving_requests = []
        self.nodes = [node_1, node_2]
        self.total_bw = bw_capacity
        
    def reset(self):
        self.serving_requests = []
        
    def add_request(self, request_obj):
        self.serving_requests.append(request_obj)
        
    def remove_request(self, request_obj):
        self.serving_requests.remove(request_obj)
        
    def remaining_bw(self): 
        # subtracting bw being used from total bw capacity
        bw_being_used = 0
        for req in self.serving_requests:
            bw_being_used += req.bw
            
        return (self.total_bw - bw_being_used)

In [4]:
class Environment:
    # requests_in_service_encoder = nn.RNN(????, 7)
    
    def __init__(self, nodes, links, request_blueprints):
        """
        nodes: list of strings where each string is just a name or identifier of a node
        links: list of tuples where in tuple t, t[0] is first node, t[1] is another node, and t[2] is bw capacity of the link
        request_blueprints: list of DeploymentRequest objects
        """
        self.nodes = nodes
        self.links = {}
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.request_blueprints = request_blueprints
        self.last_time = 0
        self.episode_timesteps = 600
        
        for link in links:
            if link[0] not in self.nodes or link[1] not in self.nodes:
                raise Exception("Node in link " + str(link) + " doesn't exist")
            
            link_obj = Link(*link)

            self.links[link[0] + link[1]] = link_obj
            self.links[link[1] + link[0]] = link_obj
            
        self.request_list = self.create_requests()
        self.request_queue = iter(self.request_list)
            
    def add_request(self, request, path=None): # we want to add this request to a link or path
        # path: a list of nodes that the request traverses including source and sink
        # if no path is specified, path is assumed to be [req.source, req.sink]
        
        if path is not None: 
            nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
            for node_pair in nodes:
                env.links[node_pair[0] + node_pair[1]].add_request(request)
        
        else:
            self.links[request.source + request.sink].add_request(request)
        
        request.accepted = True
        self.request_history.append(request)
        # print(self.links[request.source + request.sink])
    
    def reset(self):
        for link in self.links.values():
            link.reset()
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.last_time = 0
        self.request_list = env.create_requests()
        self.request_queue = iter(self.request_list)
        
        return env.get_encoding()
        
    def reward(self, request, decision):
        base_rate = 1         # 1 when static
        type_bonus = 0.9      # 0.9 when static
        bw = request.bw
        if request.type == "elastic":
            base_rate = request.bw
            type_bonus = 1.1                # 1.1 when elastic
            bw = np.array(request.bw_dist).dot(request.distribution)
            
            
        r = bw * base_rate * request.service_time * type_bonus
        
        # if remaining bandwidth on link(s) < 0, very "bad" reward
        if request.path is not None:
            nodes = [[request.path[i], request.path[i + 1]] for i in range(len(request.path) - 1)]
            for node_pair in nodes:
                if self.links[node_pair[0] + node_pair[1]].remaining_bw() < 0:
                    return (-r * 10)
        else:
            remaining_bw = self.links[request.source + request.sink].remaining_bw()
            if remaining_bw < 0:
                return (-r * 10)
        
        if decision == "accept":
            return r
        
        if decision == "reject":
            if request.type == "static":
                return 0
            elif request.type == "elastic":
                current_sum = torch.from_numpy(np.sum(self.past_distributions, axis=0))

                average_past_distribution = current_sum / len(self.past_distributions)
                current_req_distribution = torch.tensor(request.distribution)
                
                print("elastic reject")
                print(average_past_distribution)
                print(current_req_distribution)
                
                return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))
                
                """
                past_distributions = []
                for req in self.request_history:
                    if req.request_type == "elastic":
                        past_distributions.append(req.distribution)
                
                average_past_distribution = torch.mean(past_distributions, dim=1)
                current_req_distribution = torch.tensor(request.distribution)
                
                if bool(average_past_distribution[0] < current_req_distribution[0]):
                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))
                else:
                    return 0
                """
                
    def next_req(self):
        return next(self.request_queue)
                
    def step(self, req, action):
        # what happens if we have two requests that come in on the same timestep but there is only enough bandwidth for one?
        # do we the decision on the second request with knowledge of the first request
        # essentially, after we accept the first request, will we submit an updated encoding of the network to the policy network?
 
        # actions is a Nx2 matrix where the first column in the request and second is the decision
        # decision is either "accept" or "reject"
        # this is given by our agent
                
        if action[0] > 0.5:
            # accept request
            paths = (env.search(req.source, req.sink, [], []))
            paths.sort(key=lambda x: len(x)) # sort by shortest path
            # select the path we are using
            path = paths[action[1:4].argmax()]
            
            self.add_request(req, path)
        
            reward = env.reward(req, "accept")
        elif action[0] < 0.5:
            # reject
            reward = env.reward(req, "reject")
        
        obs = env.get_encoding()
        
        done = req.arrival_time > 600
        info = None
        
        return obs, reward, done, info
        
    def update_requests(self, current_time):
        # here, we remove expired requests and update E_history based off of the request stats
        
        for request in self.request_history:
            if (request.arrival_time + request.service_time) > self.last_time and (request.arrival_time + request.service_time) < current_time:
                # request has expired, let's remove it from the links
                for link in self.links.values():
                    if request in link.serving_requests:
                        link.remove_request(request)
                        
                if request.type == "elastic":
                    time_on_higher_bw = 0
                    for scale_req in request.scale_requests:
                        time_on_higher_bw += scale_req.service_time

                    time_on_lower_bw = request.service_time - time_on_higher_bw

                    # calculate E[history]
                    request_time = np.array([time_on_lower_bw, time_on_higher_bw])
                    request_bw = request.bw
                    result = (request_time / request_time.sum()).dot(request_bw)
                    self.past_distributions.append(request_time / request_time.sum())
                    self.E_history.append(result)

    def get_encoding(self):
        links_processed = [] 
        # these will store links that we have already encoded so we don't encode them again
        
        current_encoding = []
        
        # h = torch.zeros(7) # assuming 7 for h0 size
        # last_out = None
        
        env_encoding = []
        
        next_req = self.next_req()
        while next_req.type == "scale":
            if next_req.parent_elastic.accepted:
                next_req.accepted = True # we must accept since we accepted elastic req
                self.add_request(next_req, next_req.parent_elastic.path)
            next_req = self.next_req()
            
        self.update_requests(next_req.arrival_time)
        
        for link in self.links.values():
            if link in links_processed:
                continue

                        
            # Commented because we don't want to encode any queue for phase 1
            
            # for req in link.serving_requests
                # request is [one hot source, one hot destination, bw, service time, one hot type]
                
                # one_hot_source = nn.functional.one_hot(torch.tensor([self.nodes.index(req.source)]), num_classes=len(self.nodes))
                # one_hot_dest   = nn.functional.one_hot(torch.tensor([self.nodes.index(req.sink)]), num_classes=len(self.nodes))

                # req_tensor = torch.Tensor([]) # mismatched dimensions??!
                # last_out, h = self.requests_in_service_encoder(req_tensor, h)

            # current_encoding.append(torch.cat(torch.Tensor([link.remaining_bw]), last_out))
            # torch.stack(current_encoding)
            
            # check implementation later
            
            env_encoding.append(link.remaining_bw())
            
            links_processed.append(link)
            
        return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(env.nodes)), next_req
    
    def create_requests(self):
        requests = []
        
        for request_type in self.request_blueprints:
            arrival_times = []
            service_times = []
            last_arrival = 0
        
            while last_arrival < self.episode_timesteps: # we want to generate requests till we reach episode end
                last_arrival += np.random.exponential(request_type.arrival_rate)
                arrival_times.append(last_arrival)
                                
            for _ in arrival_times:
                service_times.append(np.random.exponential(request_type.service_rate))
                
            for arrival_time, service_time in zip(arrival_times, service_times):
                # start creating requests
                
                new_request = Request(request_type.type, service_time, arrival_time, request_type.source, request_type.sink, request_type.bw[0], request_type.distribution, bw_dist=request_type.bw)
                requests.append(new_request)
                
                if request_type.type == "elastic": 
                    # we will start with the first bandwidth element as starting bw
                    # WE ASSUME that bw[0] < bw[1]
                    timesteps_from_deployment = 0
                    current_bw = request_type.bw[0]
                    while timesteps_from_deployment < service_time:
                        if current_bw == request_type.bw[0]:
                            # we want to generate a scale request to increase bw
                            scale_bw = request_type.bw[1] - current_bw
                            scale_service_time = np.random.exponential(request_type.switch_rate[1])
                            scale_request = Request("scale", scale_service_time, \
                                                    arrival_time + timesteps_from_deployment, request_type.source, \
                                                   request_type.sink, scale_bw, parent_elastic=new_request)
                            requests.append(scale_request)
                            new_request.add_scale_request(scale_request)
                            
                            timesteps_from_deployment += scale_service_time
                            current_bw = request_type.bw[1] # request_type.bw[0] + scale_bw
                        elif current_bw == request_type.bw[1]:
                            # we want to go to lower bw and spend some time there
                            time_spent_on_lower_bw = np.random.exponential(request_type.switch_rate[0])
                            timesteps_from_deployment += time_spent_on_lower_bw
                            current_bw = request_type.bw[0]
                            
        # sort requests by arrival time
        requests.sort(key=lambda x: x.arrival_time)
        return requests
    
    def search(self, source, dest, visited_a, paths):
        visited_a.append(source)
        # print(visited_a)

        for link in set(env.links.values()):
            visited = visited_a.copy()
            if source in link.nodes:
                if dest in link.nodes:
                    visited.append(dest)
                    paths.append(visited)

                x = link.nodes.copy()
                x.remove(source)
                if x[0] not in visited:
                    self.search(x[0], dest, visited.copy(), paths)
        return paths

In [5]:
env = Environment(["a", "b", "c", "d", "e", "f"], [["a", "b", 10], ["a", "c", 10], ["b", "d", 10], \
                                                   ["c", "d", 20], ["c", "e", 10], ["d", "f", 10], \
                                                   ["e", "f", 10]], \
                  [RequestType("static", [2], 0.5, 0.75, "a", "b", [1]), \
                  RequestType("static", [8], 1, 1.5, "a", "b", [1]), \
                  RequestType("elastic", [4, 9], 1, 1.5, "a", "b", [0.8, 0.2], switch_rate=[0.08, 0.02]), \
                  RequestType("static", [1], 1, 1.5, "c", "d", [1]), \
                  RequestType("static", [7], 0.5, 0.75, "c", "d", [1]), \
                  RequestType("elastic", [3, 13], 2, 3, "c", "d", [0.9, 0.1], switch_rate=[0.09, 0.01]), \
                  RequestType("static", [3], 0.5, 0.75, "e", "f", [1]), \
                   RequestType("static", [6], 1, 1.5, "e", "f", [1]), \
                    RequestType("elastic", [5, 8], 2, 3, "e", "f", [0.7, 0.3], switch_rate=[0.07, 0.03])])


                # self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None

In [6]:
def policy(env_encoding, next_req_encoding, next_req_obj):    
    # find all paths between source and sink
    paths = (env.search(next_req_obj.source, next_req_obj.sink, [], []))
    paths.sort(key=lambda x: len(x)) # sort by shortest path
    selection = 0
    for path in paths:
        # check if this path works
        works = True
        nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
        for node_pair in nodes:
            if env.links[node_pair[0] + node_pair[1]].remaining_bw() < next_req_obj.bw:
                works = False
                
        if works:
            selection = paths.index(path)
            selection_one_hot = nn.functional.one_hot(torch.tensor([selection]), num_classes=3).flatten()
            next_req_obj.path = path
            return torch.cat([torch.tensor([1]), selection_one_hot])
        
    return torch.cat([torch.tensor([0]), torch.tensor([0,0,0])])

In [7]:
total_reward = 0
env_encoding, next_req_encoding, next_req_obj = env.reset()
done = False

while not done:
    decision = policy(env_encoding, next_req_encoding, next_req_obj)
    
    obs, reward, done, info = env.step(next_req_obj, decision)
    env_encoding, next_req_encoding, next_req_obj = obs
    
    print(env_encoding)
    print(reward)
    total_reward += reward

print(total_reward)

<ipython-input-4-a4a06e4edd3d>:217: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(env.nodes)), next_req
/usr/local/lib/python3.9/site-packages/torch/nn/functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor([10, 10, 10, 20, 10, 10,  4])
0.7740449151031302
tensor([10, 10, 10, 13, 10, 10, 10])
2.9645668797076885
tensor([10, 10, 10, 13, 10, 10, 10])
0.013880472646643252
tensor([10, 10, 10, 12, 10, 10, 10])
2.658561339425948
tensor([10, 10, 10, 19, 10, 10, 10])
0.6224418208141843
tensor([10, 10, 10, 12, 10, 10, 10])
4.701325169685734
tensor([ 2, 10, 10, 12, 10, 10, 10])
7.588408043297075
tensor([ 2, 10, 10,  5, 10, 10, 10])
2.0426803220146432
tensor([ 2, 10, 10,  5, 10, 10,  7])
0.6859819725341912
tensor([ 2, 10, 10,  2, 10, 10,  7])
16.314834031815355
tensor([ 0, 10, 10,  9, 10, 10, 10])
4.879915332835565
tensor([ 8, 10, 10, 16, 10, 10, 10])
0.8196572787787993
tensor([ 6, 10, 10,  6, 10, 10, 10])
0.5283911630189264
tensor([ 6, 10, 10, 16, 10, 10,  5])
72.98922140425933
tensor([ 6, 10, 10, 15, 10, 10,  5])
2.062484374719681
tensor([ 4, 10, 10, 15, 10, 10,  5])
0.7212036077019937
tensor([ 2, 10, 10, 18, 10, 10,  5])
0.6283945175664196
tensor([ 2, 10, 10, 18, 10, 10,  5])
1.4629472659464

tensor([ 2, 10, 10,  0, 10, 10,  1])
0.2808824677141651
tensor([10, 10, 10,  0, 10, 10, 10])
0
tensor([10, 10, 10, 10, 10, 10,  4])
10.052652605968534
tensor([10, 10, 10,  3, 10, 10,  4])
4.457986143683399
tensor([5, 5, 5, 6, 5, 5, 4])
165.73357374278805
tensor([5, 5, 5, 6, 5, 5, 4])
0
tensor([ 1,  5,  5, 20,  5,  5,  4])
38.85622237430328
tensor([ 1,  5,  5, 20,  5,  5,  4])
0
tensor([-4,  1,  1, 16,  5,  5,  4])
18.061699177839483
tensor([1, 1, 1, 9, 5, 5, 4])
3.294336958939659
tensor([-2, -7, -7,  4,  2,  2,  4])
0
tensor([-2, -2, -2,  9,  2,  2,  1])
0.7121191430661612
tensor([-2, -2, -2,  9,  2,  2,  1])
-6.503329963030068
tensor([1, 1, 1, 9, 5, 5, 4])
0.5051806637809139
tensor([1, 1, 1, 9, 5, 5, 1])
1.5222217802928275
tensor([-4,  1,  1,  6,  2,  2,  1])
0.47699572091170955
tensor([-4,  1,  1, 13,  2,  2,  1])
-8.174299148065344
tensor([ 1,  1,  1, 13,  2,  2,  1])
-10.236669036972874
tensor([ 1, -4, -4,  8,  2,  2,  1])
0
tensor([-2,  2,  2, 20,  2,  2,  1])
0
tensor([ 1,  3,  3

tensor([ 6, 10, 10,  4,  1,  1,  1])
1.4547913104880936
tensor([ 6, 10, 10,  4,  1,  1,  1])
0
tensor([ 2, 10, 10,  4,  1,  1,  1])
2.6262927758347097
tensor([-4, 10, 10, 11,  1,  1,  1])
0
tensor([6, 8, 8, 9, 1, 1, 1])
0.7325166992702319
tensor([6, 0, 0, 1, 1, 1, 7])
1.7606247688862575
tensor([6, 0, 0, 1, 1, 1, 7])
0
tensor([6, 0, 0, 1, 1, 1, 7])
0
tensor([10, 10, 10, 20, 10, 10, 10])
0
tensor([ 8, 10, 10, 20, 10, 10, 10])
1.189602673416079
tensor([ 0, 10, 10, 20, 10, 10, 10])
10.545185235238401
tensor([ 0, 10, 10, 13, 10, 10, 10])
3.3857305148383174
tensor([ 0, 10, 10, 13, 10, 10,  7])
0.6889469958207034
tensor([ 0, 10, 10, 12, 10, 10,  7])
0.3801137621036265
tensor([ 0, 10, 10,  5, 10, 10, 10])
8.30722298246526
tensor([ 2, 10, 10, 12, 10, 10, 10])
1.2415240397434653
tensor([ 2,  1,  1,  3, 10, 10, 10])
28.63602370465189
tensor([ 2,  6,  6,  8, 10, 10, 10])
0.09946351910184946
tensor([ 2,  6,  6,  8, 10, 10,  7])
3.437127750940345
tensor([ 0,  1,  1,  3, 10, 10,  7])
0.54353376935201

tensor([6, 1, 1, 6, 5, 5, 1])
0
tensor([10,  1,  1,  6,  5,  5,  1])
0.09468670583533334
tensor([2, 1, 1, 6, 5, 5, 1])
13.277236675733537
elastic reject
tensor([0.6128, 0.3872], dtype=torch.float64)
tensor([0.8000, 0.2000])
tensor([ 2, 10, 10, 15,  5,  5,  1])
-32.39744567548101
tensor([ 2, 10, 10,  2,  5,  5,  1])
3.9061710426737464
tensor([ 2, 10, 10, 12,  5,  5,  4])
0.25352792411060965
tensor([ 2, 10, 10, 12,  2,  2,  4])
0.22297984361871262
tensor([ 2, 10, 10, 20, 10, 10,  4])
0
tensor([10,  2,  2, 12, 10, 10,  4])
13.822113027386926
tensor([10,  2,  2, 12, 10, 10,  4])
0.08520461522231111
tensor([10,  2,  2,  6,  4,  4,  4])
5.606776413924513
tensor([10,  2,  2,  6,  4,  4,  4])
0
tensor([10,  2,  2,  6,  4,  4,  4])
0
tensor([10, 10, 10, 20, 10, 10,  4])
0.6209125607793251
tensor([10, 10, 10, 17, 10, 10,  4])
17.58648963248928
tensor([10, 10, 10, -1,  2,  2, 10])
13.115030426778358
tensor([10, 10, 10, 12,  5,  5,  7])
1.5881151531717022
tensor([10, 10, 10, 12,  5,  5,  4])
0.361

...

tensor([ 6,  6,  6,  9, 10, 10,  4])
0.4091172797427819
tensor([ 6, 10, 10, 12, 10, 10, 10])
0.2663460517115713
tensor([ 6, 10, 10, 13, 10, 10, 10])
0.6011834925711175
tensor([10, 10, 10,  6, 10, 10, 10])
1.2470680503231606
tensor([ 8, 10, 10,  6, 10, 10, 10])
1.2422556237227482
tensor([ 1,  3,  3, 20, 10, 10, 10])
4.724882925027049
elastic reject
tensor([0.6739, 0.3261], dtype=torch.float64)
tensor([0.8000, 0.2000])
tensor([ 1,  3,  3, 20, 10, 10, 10])
-27.806428721551022
tensor([ 1,  3,  3, 20, 10, 10, 10])
0.17220194504037384
elastic reject
tensor([0.6739, 0.3261], dtype=torch.float64)
tensor([0.8000, 0.2000])
tensor([10, 10, 10, 20, 10, 10, 10])
-32.40118742433124
tensor([10, 10, 10, 13, 10, 10, 10])
1.6883620425372168
tensor([10, 10, 10, 13, 10, 10,  7])
0.3345299345941569
tensor([10, 10, 10, 13, 10, 10, 10])
5.636310776700358
tensor([10, 10, 10,  6, 10, 10, 10])
1.6168210065279152
tensor([10, 10, 10, 13, 10, 10, 10])
0.48084112155852055
tensor([10, 10, 10, 13, 10, 10, 10])
0.0007

tensor([ 6, 10, 10,  3,  0,  0,  2])
0
tensor([10, 10, 10, -3, -6, -6,  5])
0
tensor([ 1, 10, 10,  0, -3, -3,  2])
69.62195785116512
tensor([ 6, 10, 10,  3,  0,  0,  2])
0
tensor([ 6, 10, 10,  5,  2,  2,  5])
0
tensor([ 6, 10, 10,  4,  2,  2,  5])
1.8746161163332171
tensor([ 6, 10, 10,  7,  5,  5,  2])
0.31792820183637044
tensor([ 2, 10, 10, 10,  5,  5,  5])
6.986234170153314
tensor([2, 1, 1, 1, 5, 5, 5])
6.9137386626935164
tensor([-3,  6,  6,  6,  5,  5,  2])
0
tensor([ 6,  2,  2, -1,  2,  2,  5])
26.804529336632132
tensor([ 6,  1,  1, -9,  5,  5,  5])
-154.9872038250761
tensor([ 1,  6,  6,  4,  2,  2, -6])
166.48950696610297
tensor([6, 6, 6, 4, 2, 2, 0])
-22.79931093886114
elastic reject
tensor([0.6733, 0.3267], dtype=torch.float64)
tensor([0.7000, 0.3000])
tensor([ 6,  6,  6,  7,  5,  5, -3])
-24.24263877918718
tensor([ 6,  1,  1,  2,  5,  5, -6])
0.0023454648911491694
tensor([ 6,  1,  1,  1,  5,  5, -3])
2.9637366734038117
tensor([ 1, 10, 10, 10,  5,  5,  5])
-33.00821253422758
ten

tensor([ 6,  6,  6, 14, 10, 10, 10])
0
tensor([ 4,  1,  1,  9, 10, 10, 10])
1.3802589565758814
tensor([ 4, 10, 10, 11, 10, 10, 10])
3.133303566014454
tensor([ 4, 10, 10, 11, 10, 10,  5])
10.97449265975809
tensor([-1, 10, 10, 11, 10, 10,  2])
0.154926601517345
tensor([ 4, 10, 10,  8,  7,  7,  2])
2.7624582033992997
tensor([ 0, 10, 10,  8,  7,  7,  5])
112.75519786377816
tensor([ 0, 10, 10,  8,  7,  7, 10])
2.161744161358523
tensor([ 0, 10, 10, 15,  7,  7,  2])
24.077539934661317
tensor([-4, 10, 10, 12,  4,  4,  5])
1.3811738052180138
tensor([ 6,  6,  6, 11,  7,  7,  5])
7.114818478240812
tensor([ 6,  6,  6, 14, 10, 10, 10])
0
tensor([ 6, 10, 10, 18, 10, 10, 10])
0.12419994101509128
tensor([ 6, 10, 10, 20, 10, 10, 10])
0.7185628339396675
tensor([ 6, 10, 10, 13, 10, 10, 10])
2.54375713624373
tensor([ 6, 10, 10, 13, 10, 10,  4])
0.7150691667533683
tensor([ 6, 10, 10,  8,  5,  5, 10])
154.3271369233384
tensor([ 6, 10, 10,  5,  2,  2,  7])
1.399001899848889
tensor([ 2, 10, 10,  5,  2,  2,  7

tensor([10,  0,  0,  1,  4,  4,  2])
2.4409386443354966
tensor([10,  0,  0,  1,  4,  4,  5])
0
tensor([10,  2,  2,  9,  7,  7,  7])
0
tensor([10,  2,  2,  9,  7,  7,  4])
0.531336378827573
tensor([10,  2,  2,  2,  7,  7, 10])
7.374432377489765
tensor([ 6,  2,  2,  2,  7,  7, 10])
18.94476660924405
tensor([6, 2, 2, 5, 3, 3, 3])
2.965165461590208
tensor([4, 2, 2, 5, 3, 3, 3])
0.5755139571434686
tensor([4, 2, 2, 5, 3, 3, 0])
0.36560826867388546
tensor([4, 2, 2, 2, 0, 0, 3])
2.548003184240283
tensor([ 6,  2,  2,  2,  7,  7, 10])
0.6481182607594833
tensor([ 6,  2,  2,  2,  7,  7, 10])
0.04604021707937615
tensor([10,  2,  2,  9,  7,  7,  4])
6.658998153295843
tensor([10, 10, 10, 10,  7,  7,  4])
3.0318806489978125
tensor([10, 10, 10,  7,  4,  4,  4])
2.5485892873197034
tensor([10, 10, 10,  7,  4,  4,  1])
2.330047729939592
tensor([10, 10, 10, 20, 10, 10,  1])
1.3466510969294128
tensor([10, 10, 10, 14,  4,  4,  1])
5.400132762534066
tensor([ 2, 10, 10, 14,  4,  4,  1])
1.3531316985097182
elas

tensor([8, 8, 8, 9, 4, 4, 2])
0.9299956454573842
tensor([ 6,  8,  8, 15, 10, 10,  5])
0.9124390333055066
tensor([ 4,  8,  8, 15, 10, 10,  5])
0.26782343720572294
tensor([4, 8, 8, 9, 4, 4, 5])
12.91018153134683
tensor([10, 10, 10, 11,  4,  4,  5])
1.7856272842102001
tensor([10, 10, 10,  1,  4,  4,  5])
0
tensor([ 8, 10, 10, 11,  4,  4,  5])
1.3762666067188454
tensor([ 8, 10, 10,  4,  4,  4,  5])
6.930331060512358
tensor([ 8, 10, 10,  4,  4,  4,  5])
0.20381739413244712
tensor([ 8, 10, 10,  4,  4,  4,  5])
0.5834276906466995
tensor([10, 10, 10,  3,  4,  4,  5])
0.7348021134280539
tensor([ 8, 10, 10,  3,  4,  4,  5])
3.0216086313288186
tensor([ 8, 10, 10, -7,  4,  4,  5])
0.7738153938840013
tensor([ 1,  3,  3, 10,  4,  4,  5])
18.338909312723484
tensor([ 1,  3,  3,  3,  4,  4, 10])
5.160688039319132
tensor([ 1,  3,  3, -9,  4,  4, 10])
104.74320665162652
tensor([ 1,  3,  3, -9,  4,  4, 10])
-19.185298033161363
tensor([ 1,  3,  3,  1,  4,  4, 10])
0.005271331251055704
elastic reject
tensor